In [2]:
from qutip import *
from qutip.qip.operations import rx, ry, rz
import numpy as np

In [3]:
# dimension of our Fock Space
cutoff = 10

# identity operator for one qubit 
I2 = identity(2)

# annihilation and identity operator in Fock Space
a = destroy(cutoff)
Ib = identity(cutoff)

# define Pauli matrices and constants
σ_x = sigmax()
σ_y = sigmay()
σ_z = sigmaz()
π = np.pi

# Gates

## Native Gates

$$ \huge D(\alpha) = e^{\alpha a^{\dagger} - \alpha^{*} a}$$

In [4]:
# displacement operator 
def D(α):
    return (α*a.dag() - np.conj(α)*a).expm()

$$ \huge D_{c}(\alpha, \beta) = D(\alpha) \otimes |0 \rangle \langle 0 | + D(\beta) \otimes |1 \rangle \langle 1 | $$

In [5]:
def Dc(α, β): 
    return tensor(D(α), basis(2,0)*basis(2,0).dag()) + tensor(D(β), basis(2,1)*basis(2,1).dag())

$$ \huge \text{CP} = e^{-i \frac{\pi}{2} a^{\dagger} a \otimes Z } $$

In [6]:
def CP():
    return (-1j*(np.pi/2)*tensor(a.dag()*a,sigmaz())).expm()

$$ \huge \text{BS}^{(i,j)}(\theta, \varphi) = e^{-i \frac{\theta}{2} \left[e^{i \varphi}a^{\dagger}_i \otimes \mathbb{1} a_j \otimes \mathbb{1} + e^{-i \varphi} a_i \otimes \mathbb{1} \otimes a^{\dagger}_j \otimes \mathbb{1} \right]}$$

In [7]:
# beamsplitter gate
def BS(θ,ϕ): 
    # cavity_1 ⊗ qubit_1 ⊗ cavity_2 ⊗ qubit_2
    term = tensor([a.dag(), I2, a, I2])
    phase = np.exp(1j*ϕ) 
    return (-1j*(θ/2)*(phase*term + np.conj(phase)*term.dag())).expm()

## Native-Like Gates and Special Cases

In [8]:
# generate θ (random float between 0 and 1) and define α as function of θ
θ_list = [np.random.rand() for i in range(2)]
α = np.sqrt(θ_list[0]/4)
β = np.sqrt(θ_list[1]/4)

# define Hadamard gate
H = Qobj([[1,1],[1,-1]],dims=[[2],[2]])/np.sqrt(2)

# define |+><+| and |-><-|
plus = H*basis(2, 0)
minus = H*basis(2, 1)
plus_projector = plus*plus.dag()
minus_projector = minus*minus.dag()

# define |+i><+i| and |-i><-i| 
plus_i = (basis(2, 0) + 1j*basis(2, 1))/np.sqrt(2)
minus_i = (basis(2, 0) - 1j*basis(2, 1))/np.sqrt(2)
plus_i_projector = plus_i * plus_i.dag()
minus_i_projector = minus_i * minus_i.dag()

$$ \LARGE \text{CP}_Z = e^{-i \frac{\pi}{2} a^{\dagger} a \otimes Z} $$

In [9]:
display(CP() == (-1j*π/2*tensor(a.dag()*a, σ_z)).expm())
CPZ = CP()

True

$$ \LARGE \text{CP}_X = H \text{CP}_Z H = e^{-i \frac{\pi}{2} a^{\dagger} a \otimes X} $$

In [10]:
display(tensor(Ib, H)*CP()*tensor(Ib, H) == (-1j*π/2*tensor(a.dag()*a, σ_x)).expm())
CPX = (-1j*π/2*tensor(a.dag()*a, σ_x)).expm()

True

$$ \LARGE \text{CP}_Y = R_X \left(-\frac{\pi}{2} \right) \text{CP}_Z R_X \left(\frac{\pi}{2} \right) =  e^{-i \frac{\pi}{2} a^{\dagger} a \otimes Y} $$

In [11]:
display(tensor(Ib, rx(-π/2))*CP()*tensor(Ib, rx(π/2)) == (-1j*π/2*tensor(a.dag()*a, σ_y)).expm())
CPY = (-1j*π/2*tensor(a.dag()*a, σ_y)).expm()

True

$$ \Large D_{cZ}(\alpha, \beta) = D(\alpha) \otimes |0 \rangle \langle 0 | + D(\beta) \otimes |1 \rangle \langle 1 |$$

In [12]:
display(Dc(α, β) == tensor(D(α), basis(2,0)*basis(2,0).dag()) + tensor(D(β), basis(2,1)*basis(2,1).dag()))
DcZ = lambda α,β: Dc(α, β)

True

$$ \Large D_{cX}(\alpha, \beta) = D(\alpha) \otimes |+ \rangle \langle + | + D(\beta) \otimes |- \rangle \langle - |$$

In [13]:
display(tensor(Ib,H)*Dc(α,β)*tensor(Ib,H) == tensor(D(α), plus_projector) + tensor(D(β), minus_projector))
DcX = lambda α,β: tensor(Ib,H)*Dc(α,β)*tensor(Ib,H)

True

$$ \Large D_{cY}(\alpha, \beta) = D(\alpha) \otimes |+i\rangle \langle+i| + D(\beta) \otimes |-i\rangle \langle-i|$$

In [14]:
display(tensor(Ib,rx(-π/2))*Dc(α,β)*tensor(Ib,rx(π/2)) == tensor(D(α), plus_i_projector) + tensor(D(β), minus_i_projector))
DcY = lambda α,β: tensor(Ib,rx(-π/2))*Dc(α,β)*tensor(Ib,rx(π/2))

True

$$ \Large D_{Z}(\alpha, -\alpha) = \text{CP}_{Z}^{\dagger} D(-i \alpha) \text{CP}_{Z} = e^{\left(\alpha a^{\dagger} - \alpha^{*} a \right) \otimes Z} $$

In [15]:
display(CPZ.dag() * tensor(D(-1j*α), I2) * CPZ == (tensor(α*a.dag() - α*a, σ_z)).expm())
DsZ = lambda α: (tensor(α*a.dag() - α*a, σ_z)).expm()

True

$$ \Large D_{X}(\alpha, -\alpha) =\text{CP}_{X}^{\dagger} D(-i \alpha) \text{CP}_{X} = e^{\left(\alpha a^{\dagger} - \alpha^{*} a \right) \otimes X} $$

In [16]:
display(CPX.dag()*tensor(D(-1j*α), I2)*CPX == (tensor(α*a.dag() - α*a, σ_x)).expm())
DX = lambda α: (tensor(α*a.dag() - α*a, σ_x)).expm()

True

$$ \Large D_{Y}(\alpha, -\alpha) =\text{CP}_{Y}^{\dagger} D(-i \alpha) \text{CP}_{Y} = e^{\left(\alpha a^{\dagger} - \alpha^{*} a \right) \otimes Y} $$

In [17]:
display(CPY.dag() * tensor(D(-1j*α), I2)*CPY == (tensor(α*a.dag() - α*a, σ_y)).expm())
DY = lambda α: (tensor(α*a.dag() - α*a, σ_y)).expm()

True

# $$ D_{XYZ} = e^{ \left(\alpha a^{\dagger} - \alpha^{*} a \right) \otimes \left( X_1 X_2 + Y_1 Y_2 + Z_1 Z_2 \right)} $$

$$ \Large D_{ZZ}(\alpha) = \text{CP}^{\dagger}_Z D_{cZ}(-i\alpha, +i \alpha) \text{CP}_Z = e^{\left(\alpha a^{\dagger} - \alpha^{*} a  \right) \otimes Z_1 Z_2} $$

In [38]:
CPZ12 = (-1j*(np.pi/2)*tensor(a.dag()*a, I2, Ib, σ_z)).expm()
display(CPZ12.dag()*tensor(DcZ(-1j*α, 1j*α),Ib,I2)*CPZ12 == (tensor(α*a.dag() - np.conj(α)*a, σ_z, Ib, σ_z)).expm())
DZZ = lambda α: (tensor(α*a.dag() - np.conj(α)*a, σ_z, Ib, σ_z)).expm()

True

$$ \Large D_{XX}(\alpha) = \text{CP}^{\dagger}_X D_{cX}(-i\alpha, +i \alpha) \text{CP}_X = e^{\left(\alpha a^{\dagger} - \alpha^{*} a  \right) \otimes X_1 X_2}$$

In [39]:
CPX12 = (-1j*(np.pi/2)*tensor(a.dag()*a, I2, Ib, σ_x)).expm()
display(CPX12.dag()*tensor(DcX(-1j*α, 1j*α),Ib,I2)*CPX12 == (tensor(α*a.dag() - np.conj(α)*a, σ_x, Ib, σ_x)).expm())
DXX = lambda α: (tensor(α*a.dag() - np.conj(α)*a, σ_x, Ib, σ_x)).expm()

True

$$ \Large D_{YY}(\alpha) = \text{CP}^{\dagger}_Y D_{cY}(-i\alpha, +i \alpha) \text{CP}_Y = e^{\left(\alpha a^{\dagger} - \alpha^{*} a  \right) \otimes Y_1 Y_2}$$

In [41]:
CPY12 = (-1j*(np.pi/2)*tensor(a.dag()*a, I2, Ib, σ_y)).expm()
display(CPY12.dag()*tensor(DcY(-1j*α, 1j*α),Ib,I2)*CPY12 == (tensor(α*a.dag() - np.conj(α)*a, σ_y, Ib, σ_y)).expm())
DYY = lambda α: (tensor(α*a.dag() - np.conj(α)*a, σ_y, Ib, σ_y)).expm()

True

$$ \Large D_{XYZ}(\alpha) = D_{XX}(\alpha) D_{YY} (\alpha) D_{ZZ}(\alpha) = e^{\left(\alpha a^{\dagger} - \alpha^{*} a  \right) \otimes \left( X_1 X_2 + Y_1 Y_2 + Z_1 Z_2 \right)}$$

In [42]:
# define XX + YY + ZZ 
sum_XYZ = tensor(σ_x, Ib, σ_x) + tensor(σ_y, Ib, σ_y) + tensor(σ_z, Ib, σ_z)
display(DXX(α)*DYY(α)*DZZ(α) == (tensor(α*a.dag() - np.conj(α)*a, sum_XYZ)).expm())
DXYZ = lambda α: (tensor(α*a.dag() - np.conj(α)*a, sum_XYZ)).expm()

True

# Synthesized Gates

$$ \Large R_{ZZ}(\theta) = e^{- i \frac{\theta}{2} Z_1 \otimes Z_2} $$

In [ ]:
θ = np.random.rand()
α = np.sqrt(θ/4)

U1zz = tensor(Dc(-α,α),Ib,I2)
U2zz = BS(π,π/2)*tensor(Ib,I2,Dc(1j*α,-1j*α))*BS(π,-π/2)#.permute([0,3,2,1])

Uzz = U2zz.dag()*U1zz.dag()*U2zz*U1zz

P = tensor(basis(cutoff,0),I2,basis(cutoff,0),I2)
P.dag()*Uzz*P == (-1j*θ*tensor(sigmaz(),sigmaz())/2).expm()

$$ \Large \text{CNOT} = e^{i \frac{\pi}{4} \left[I_1 - Z_1 \right] \otimes \left[I_2 - X_2 \right]} $$

In [ ]:
### CNOT
α = np.sqrt(π/2)
U1_cnot = tensor([Dc(0, α),Ib, I2])
U2_cnot = BS(π,π/2)*tensor([Ib, I2, DcX(0, 1j*α)])*BS(π,-π/2)

U_cnot = (U2_cnot.dag()*U1_cnot.dag()*U2_cnot*U1_cnot)

P = tensor(basis(cutoff,0).dag(),I2,basis(cutoff,0).dag(),I2)
P*U_cnot*P.dag()

$$ \Large \text{TOFFOLI} = e^{i \frac{\pi}{8} \left[I_1 - Z_1 \right] \otimes \left[I_2 - Z_2 \right] \otimes \left[I_3 - X_3 \right]} $$

In [ ]:
# distance by which we displace oscillator in phase space
α = np.sqrt(π/2)

cp = tensor(CPZ,Ib,I2)

d1 = tensor([Ib,I2,Dc(0,1j*α)])
d2 = tensor([Ib,I2,CD(0,α)])

In [ ]:
tensor([Ib,I2, Dc(0, α)]) == (tensor(Ib, I2, α*a.dag() - np.conj(α)*a, (I2-σ_z)/2)).expm()

In [ ]:
BS(π, π/2)*tensor([Ib,I2, Dc(0, α)])*BS(π,-π/2) == (tensor(α*a.dag() - np.conj(α)*a, I2, Ib, (I2-σ_z)/2)).expm()

In [ ]:
# define a controlled-parity opeartor that flips qubit_1 depending on parity of photons in cavity
# (flip if odd and no flip if even) conditioned upon 
cp = tensor(CP(),I2)

# asymmetric diplacement operators that applies D(0) (D(0)) if qubit_2 is in |0> and D(-iα/2) (D(α/2)) if qubit_2 is in |1>
d1 = tensor([Dc(0,-1j*α/2),Ib, I2]).permute([0,2,1])
d2 = tensor([Dc(0, α/2),I2]).permute([0,2,1])

# conditional-conditional displacement 
U12 = tensor([d2*cp*d1*cp.dag(),I2])

# apply D(0) if qubit_3 is in |+> and D(+iα) if qubit_3 is in |->
U3 = tensor([DCX(1j*α),I2,I2]).permute([0,3,2,1])

# gate sequence that carves out area in phase space according to CNOT conditions
Utoff = U3.dag()*U12.dag()*U3*U12

# trace out oscillator by assuming we start and end in the ground state
P = tensor(basis(cutoff,0),I2,I2,I2)
P.dag()*Utoff*P 

# Parallelized Gates

$$ \Large R_{ZZ}^{(||)}(\theta) = e^{- i \frac{\theta}{2} Z_1 \otimes Z_2} $$

In [ ]:
# define random θ and α as a function of θ
# two different phase space diagrams, each of which have 4 panels which individually enclose α/2
θ = np.random.rand()
α = np.sqrt(θ/4)/np.sqrt(2)

# note that the tensor space of the beamsplitter is cavity_1 ⊗ qubit_1 ⊗ cavity_2 ⊗ qubit_2
# the tensor space of term1 is cavity_1 ⊗ qubit_1 ⊗ cavity_2 ⊗ qubit_2
term1  = tensor(Dc(-α,α), Dc(-α,α))

# swap mode 1 and 2 before applying two conditional displacements in parallel. 
term2 = BS(π,-π/2).dag()*tensor(Dc(-1j*α,1j*α),Dc(1j*α,-1j*α))*BS(π,-π/2)

# gate sequence that carves out area in phase space according to RZZ conditions
U = term2.dag()*term1.dag()*term2*term1

# projection operator that begins in the ground state
P = tensor(basis(cutoff,0),I2,basis(cutoff,0),I2)

# trace out oscillator by assuming we start and end in the ground state
RZZ_parallel_synthesized = P.dag()*U*P 

# explicitly define RZZ gate
RZZ_direct = (-1j*θ*tensor(sigmaz(),sigmaz())/2).expm()

# check whether RZZ_direct and RZZ_parallel_sythesized are equal 
RZZ_parallel_synthesized # == RZZ_direct 

$$ \Large \text{CNOT}^{(||)} = e^{i \frac{\pi}{4} \left[I_1 - Z_1 \right] \otimes \left[I_2 - X_2 \right]} $$

In [ ]:
α = np.sqrt(π)/2
term1  = tensor([DCZ(α), DCZ(α)])
term2 = BS(π,π/2)*tensor([DCZ(-1j*α),DCZ(1j*α)])*BS(π,-π/2)
term3  = tensor([DCZ(-α), DCZ(-α)])
term4 = BS(π,π/2)*tensor([DCZ(1j*α), DCZ(-1j*α)])*BS(π,-π/2)

H = Qobj([[1,1],[1,-1]],dims=[[2],[2]])/np.sqrt(2)

# U = tensor(Ib,I2,Ib,H)*term2.dag()*term1.dag()*term2*term1*tensor(Ib,I2,Ib,H)
U = tensor(Ib,I2,Ib,H)*term4*term3*term2*term1*tensor(Ib,I2,Ib,H)

P = tensor(basis(cutoff,0).dag(),I2,basis(cutoff,0).dag(),I2)
P*U*P.dag()

$$ \Large R_{xz}^{(||)} \left( \theta \right) = e^{- i \frac{\theta}{2} \left(X_1 \otimes X_2 + Y_1 \otimes Y_2 \right ) }$$

In [ ]:
## XX+ZZ parallel

theta = np.pi/2
alpha = np.sqrt(theta/4)/np.sqrt(2)

H = Qobj([[1,1],[1,-1]],dims=[[2],[2]])/np.sqrt(2)

term1  = tensor(Ib,I2,Ib,H)*tensor(Dc(-alpha,alpha),Dc(-alpha,alpha))*tensor(Ib,I2,Ib,H)
term2 = tensor(Ib,I2,Ib,H)*BS(np.pi,np.pi/2)*tensor(Dc(-1j*alpha,1j*alpha),Dc(1j*alpha,-1j*alpha))*BS(np.pi,-np.pi/2)*tensor(Ib,I2,Ib,H)
# term3  = tensor(Ib,I2,Ib,H)*tensor(Dc(alpha,-alpha),Dc(alpha,-alpha))*tensor(Ib,I2,Ib,H)
# term4 = tensor(Ib,H,Ib,I2)*BS(np.pi,np.pi/2)*tensor(Dc(1j*alpha,-1j*alpha),Dc(-1j*alpha,1j*alpha))*BS(np.pi,-np.pi/2)*tensor(Ib,H,Ib,I2)

# term1  = tensor(Dc(-alpha,alpha),Dc(-alpha,alpha))
# term2 = BS(np.pi,np.pi/2)*tensor(Dc(-1j*alpha,1j*alpha),Dc(1j*alpha,-1j*alpha))*BS(np.pi,-np.pi/2)
# term3  = tensor(Dc(alpha,-alpha),Dc(alpha,-alpha))
# term4 = BS(np.pi,np.pi/2)*tensor(Dc(1j*alpha,-1j*alpha),Dc(-1j*alpha,1j*alpha))*BS(np.pi,-np.pi/2)

# U = tensor(Ib,I2,Ib,H)*term2.dag()*term1.dag()*term2*term1*tensor(Ib,I2,Ib,H)
U = term2.dag()*term1.dag()*term2*term1

In [ ]:
P = tensor(basis(cutoff,0).dag(),I2,basis(cutoff,0).dag(),I2)
P*U*P.dag()

In [ ]:
(-1j*(theta/2)*(tensor(sigmaz(),sigmaz()) + tensor(sigmax(),sigmax()))).expm()

In [ ]:
np.arctan(0.132/0.932)/np.pi

In [ ]:
np.sqrt(1/5)

In [ ]:
Util = P*U*P.dag()

In [ ]:
Util.dag()*Util

# $ D_{XYZ} = e^{ \left(\alpha a^{\dagger} - \alpha^{*} a \right) \otimes \left( X_1 X_2 + Y_1 Y_2 + Z_1 Z_2 \right)} $

$$ \Large CP^{(1, 1)} = e^{-i \frac{\pi}{2} a^{\dagger} a \otimes \sigma_z \otimes \mathbb{1}}$$

In [ ]:
CPX_3(1) == tensor(Ib, H, I2) * CP3(1, 'Z') * tensor(Ib, H, I2) and \
CP3(1, 'Z') == CP3(2, 'Z').permute([0, 2, 1]) and \
CPX_3(1) == CPX_3(2).permute([0, 2, 1]) and \
CP3(2, 'X') == CPX_3(2) and \
CP3(1, 'Y') == CPY_3(2).permute([0, 2, 1]) and \
CP3(2, 'X') == CPX_3(2) and \
CP3(2, 'Y') == CPY_3(2)

In [ ]:
# controlled parity gate for a Hilbert space of three subsystems 
def CP3(qubit, pauli_operator):
    """
    cavity_1 ⊗ qubit_1 ⊗ qubit_2
    """
    # specify Pauli operator
    if pauli_operator == 'X':
        σ_i = sigmax()
    elif pauli_operator == 'Y':
        σ_i = sigmay()
    elif pauli_operator == 'Z':
        σ_i = sigmaz()
    
    # specify control qubit 
    if qubit == 1: 
        return (-1j*(np.pi/2)*tensor(a.dag()*a, σ_i, I2)).expm()
    elif qubit == 2: 
        return (-1j*(np.pi/2)*tensor(a.dag()*a, I2, σ_i)).expm()
    else:
        print("Please enter valid target qubit")

$$ \Large D_c^{(1, 1)}(\alpha, \beta) = D(\alpha) \otimes |0 \rangle \langle 0 | \otimes \mathbb{1} + D(\beta) \otimes |1 \rangle \langle 1 | \otimes \mathbb{1} $$

In [ ]:
def Dc_3(α, β):
    
    """ 
    cavity_1 ⊗ qubit_1 ⊗ qubit_2
    controlled-displacement conditioned on the 1st qubit
    """
    return tensor(D(α), basis(2,0)*basis(2,0).dag(), I2) + tensor(D(β), basis(2,1)*basis(2,1).dag(), I2)

In [ ]:
def Dc4(α, β):
    """ 
    cavity_1 ⊗ qubit_1 ⊗ cavity_2 ⊗ qubit_2
    controlled-displacement conditioned on the 1st qubit
    """
    return tensor(D(α), basis(2,0)*basis(2,0).dag(), Ib, I2) + tensor(D(β), basis(2,1)*basis(2,1).dag(), Ib, I2)

$$ \Large D_{c}(\alpha, - \alpha) = \text{CP}^{\dagger} D(-i \alpha) \text{CP} $$

In [ ]:
Dc(α, -α) == CP('Z').dag() * tensor(D(-1j*α), I2) * CP('Z')

$$ \Large D_{ZZ}(\alpha) = \text{CP}^{\dagger} D_{c}(-i\alpha, +i \alpha) \text{CP} $$

In [ ]:
DZZ(α) == CP3(2, 'Z').dag() * Dc_3(-1j*α, 1j*α) * CP3(2, 'Z')

$$ \Large D_{XX}(\alpha) = \text{CP}^{\dagger} D_{c}(-i\alpha, +i \alpha) \text{CP} $$

In [ ]:
DXX(α) == CP3(2, 'X').dag() * tensor(Ib, H, I2) * Dc_3(-1j*α, 1j*α) * tensor(Ib, H, I2) * CP3(2, 'X')

In [ ]:
DYY(α) == CP3(2, 'Y').dag() * tensor(Ib, rx(-π/2), I2) * Dc_3(-1j*α, 1j*α) * tensor(Ib, rx(π/2), I2) * CP3(2, 'Y')

In [ ]:
DXYZ(α) == DXX(α) * DYY(α) * DZZ(α)

$$ \Large D_{YY}(\alpha) = \text{CP}^{\dagger} D_{c}(-i\alpha, +i \alpha) \text{CP} $$